In [3]:
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi


import numpy as np
%matplotlib inline
import hvplot.pandas
from decouple import config
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from finta import TA

import pandas_datareader.data as web
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
#from MCForecastTools import mc_simulation

style.use('ggplot')

from mc_simulation import AverageModel as am
from mc_simulation import VolatilityModel as vm
from mc_simulation import MonteCarloSimulation as mcs

#API_USERNAME = config('USER')
#API_KEY = config('KEY')

In [4]:
load_dotenv()


True

In [5]:

#Read the file in dataframe 
file_path = './Resources/AAPL_df.csv'
AAPL_df = pd.read_csv(Path(file_path),index_col='Date',parse_dates=True,infer_dateframe_format=True)

AAPL_df['actual returns'] = AAPL_df['close'].pct_change()
AAPL_df = AAPL_df.dropna()

AAPL_df.head()



TypeError: read_csv() got an unexpected keyword argument 'infer_dateframe_format'

In [ ]:
# Make X and Y Data Sets

#Read the file in dataframe 
file_path = './Resources/AAPL_df.csv'
AAPL_df = pd.read_csv(Path(file_path),index_col='Date',parse_dates=True,infer_dateframe_format=True)

AAPL_df['actual returns'] = AAPL_df['close'].pct_change()
AAPL_df = AAPL_df.dropna()

AAPL_df.head()


In [ ]:
sma_fast = 4
sma_slow = 100
ema = 50
volatility_window = 4

AAPL_df['sma_fast'] = TA.SMA(AAPL_df, sma_fast)
AAPL_df['sma_fast'] = TA.SMA(AAPL_df, sma_slow)
AAPL_df['ssma'] = TA.SSMA(AAPL_df)
AAPL_df['ema'] = TA.EMA(AAPL_df, ema)
AAPL_df['dema'] = TA.DEMA(AAPL_df)
AAPL_df['tema'] = TA.TEMA(AAPL_df)
AAPL_df['trima'] = TA.TRIMA(AAPL_df)
AAPL_df['volatility'] = AAPL_df['actual_returns'].rolling(window=volatility_window).std()

AAPL_df.dropna(inplace=True)
AAPL_df

In [ ]:
#Make X Data Set

X = AAPL_df[['sma_fast', 'sma_slow', 'ssma', 'ema', 'dema', 'tema', 'volatility']].shift().dropna().copy
X


In [ ]:
#Make Y Data Set

#Make the Buy/Sell signal and populate them with two opposing integers: +1 or -1
AAPL_df['signal'] = 0.0

#Make buy and sell signal
AAPL_df['signal'] = np.where(AAPL_df['actual_returns'] >=0, 1. -1)

y = AAPL_df['signal']

In [ ]:
#Make Train and Test Data: X_train, y_train, X_test, y_test
#offset_years as a parameter

offset_years = 8
training_begin = X.index.min()
training_end = training_begin + DateOffset(years=offset_years)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

test_begin = X.loc[training_end : ].index.min()
X_test = X.loc[test_begin : ]
y_test = y.loc[test_begin : ]


In [ ]:
#Standardizing Our Data Set

#Scaling the features Dataframes

#Make a StandardScaler instance
scaler = StandardScaler()

#Apply the scaler model to fit the X_train data set
X_scaler = scaler.fit(X_train)

#Transform the X_train and X_test Dataframes using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
#Make a Model
#Fit the Model with X_train_scaled, y_train data
#Make comparision report


from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

#Make models

#Logistic Regression model
LR_model = LogisticRegression(random_state=1)
LR_model.fit(X_train_scaled,y_train)
y_predict_test_LR = LR_model.predict(X_test_scaled)
print('Logistic Regression model')
print(classification_report(y_test,y_predict_test_LR)

#DecisionTreeClassifier model
DTC_model = DecisionTreeClassifier(random_state=1)
DTC_model.fit(X_train_scaled, y_train)
y_predict_test_DTC = DTC_model.predict(X_test_scaled) 
print('Decision Tree Classifier')
print(classification_report(y_test, y_predict_test_DTC))

# SVM model
SVM_model = SVC(random_state=1)
SVM_model.fit(X_train_scaled,y_train)
y_predict_test_SVM = SVM_model.predict(X_test_scaled)
print('SVM Classifier')
print(classification_report(y_test,y_predict_test_test_SVM)
      
# GaussianNB model
GaussianNB_model = GaussianNB()
GaussianNB_model.fit(X_train_scaled,y_train)
y_predict_test_GaussianNB = GaussianNB_model.predict(X_test_scaled)
print('GaussianNB Classifier')
print(classification_report(y_test,y_predit_test_GaussianNB)
      
#Random Forest Classifier model
RandomForestClassifier_model = RandomForestClassifier()
RandomForestClassifier_model_model.fit(X_train_scaled,y_train)
y_predict_test_GRandomForestClassifier = RandomForestClassifier_model.predict(X_test_scaled)
print('RandomForestClassifier_model')
print(classification_report(y_test,y_predit_test_RandomForestClassifier)
      

